## Sentiment Analysis with Deep Learning using BERT

### What is BERT

BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks.

For more information, the original paper can be found [here](https://arxiv.org/abs/1810.04805). 

[HuggingFace documentation](https://huggingface.co/transformers/model_doc/bert.html)

[Bert documentation](https://characters.fandom.com/wiki/Bert_(Sesame_Street) ;)

<img src="Images/BERT_diagrams.png" width="1000">

We will use the SMILE Twitter dataset.

Dataset. https://doi.org/10.6084/m9.figshare.3187909.v2_

In [1]:
#importing the required libraries
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
#loading the Smile twitter dataset and setting the index to id since it will be unique
df=pd.read_csv('Data/smile-annotations-final.csv',
              names=['id','text','category'])
df.set_index('id',inplace=True)

In [3]:
#to see the top five entry of dataset
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [4]:
#to check the first text entry
df.text.iloc[0]

'@aandraous @britishmuseum @AndrewsAntonio Merci pour le partage! @openwinemap'

In [5]:
#let's see the various emotion category
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [7]:
#removing tweet that have multiple emotions, since we will use one-hot encoding so it will be tough to one hot emotions
df=df[-df.category.str.contains('\|')]

In [8]:
#removing tweet that have no code as emotions
df=df[df.category !='nocode']

In [9]:
#we have class imbalance now
df.category.value_counts()
#just to check whether we have droped values or not properly

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [10]:
#framing a list of unique categories
possible_labels=df.category.unique()

In [11]:
#building the dictionary with key as emotions and values a there unique number index
label_dict={}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label]=index

In [12]:
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [13]:
#Adding the label column to our dataset with each category alonside with their index values
df['label']=df.category.replace(label_dict)
df.head(10)

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0
614499696015503361,Lucky @FitzMuseum_UK! Good luck @MirandaStearn...,happy,0
613601881441570816,Yr 9 art students are off to the @britishmuseu...,happy,0
613696526297210880,@RAMMuseum Please vote for us as @sainsbury #s...,not-relevant,1
610746718641102848,#AskTheGallery Have you got plans to privatise...,not-relevant,1


In [14]:
#to prepare the training and validation data
from sklearn.model_selection import train_test_split

In [15]:
#since the classes are imbalance so we will split the data in stratified way according to the classes
X_train,X_val,y_train,y_val=train_test_split(
    df.index.values,
    df.label.values,
    test_size=0.15,
    random_state=17,
    stratify=df.label.values
)
#spliting the data according to class labels so that train and test data will have all the classes distribution

In [16]:
df['data_type']=['not_set']*df.shape[0]
#adding an extra column so that if the sample is in train set or test set

In [17]:
df.head()

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,not_set


In [19]:
df.loc[X_train, 'data_type']='train'
df.loc[X_val, 'data_type']='val'
#to set the data_type whether it belongs to train data or test data

In [20]:
df.groupby(['category','label','data_type']).count()
#to check that the stratified split occurs or not

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

In [21]:
#loading tokenizer and encoding the data
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
#Tokenizing the words or converting the text into numeric value

In [22]:
tokenizer=BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)
#it will take the pretrained bert tokenizer which tokenizes on all the lower cases so we convert the data to lower case
#using do_lower_case attribute

In [23]:
encoded_data_train=tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)
#add_speical tokens will tell the tokenizer where the sentence ended
#attenion mask will tell the tokenizer where the original sentence is and where the padded values are
#since sentences are all different length so we have to made them to equal length by padding to make the size of input fix
#since we are using pytorch we will be returing torch tensors

#similar for the validation data
encoded_data_val=tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

#it will return a dictionary
input_ids_train=encoded_data_train['input_ids']
attention_masks_train=encoded_data_train['attention_mask']
labels_train=torch.tensor(df[df.data_type=='train'].label.values)

#since only this will be required for the input
input_ids_val=encoded_data_val['input_ids']
attention_masks_val=encoded_data_val['attention_mask']
labels_val=torch.tensor(df[df.data_type=='val'].label.values)

In [25]:
dataset_train=TensorDataset(input_ids_train,attention_masks_train,labels_train)
dataset_val=TensorDataset(input_ids_val,attention_masks_val,labels_val)
#making our dataset using TensorDataset and passing the above three arguments

In [26]:
len(dataset_train)

1258

In [27]:
len(dataset_val)

223

In [28]:
from transformers import BertForSequenceClassification
#importing the bert sequence library,each tweet as one sequence

In [29]:
#importing the pretrained model
model=BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False
)
#this is the final tunning step where we wil specify the number of classes in the output
#using num_lables
#setting hidden states output False will won't show the previous layers output
#setting output attentions to false will not emphazize on one output

In [30]:
#creating the data loaders which will help us to iterate over the dataset easily
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [31]:
batch_size=4   #32

dataloader_train=DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)
#using Random Sampler for train set since it should be shuffled randomly

dataloader_val=DataLoader(
    dataset_val,
    sampler=SequentialSampler(dataset_val),
    batch_size=32
)

#setting batch size to 32 back since it won't require much computation in val set 
#since there in no back propogation calculation unlike during training
#using Sequnece Sampler for validation set since it won't be matter to randomly shuffled it we can take it in sequnece also

In [32]:
#setting up the optimizer and Scheduler
#adamW is adam with weight decay
from transformers import AdamW, get_linear_schedule_with_warmup

In [33]:
optimizer=AdamW(
    model.parameters(),
    lr=1e-5,   # 2e-5>5e-5
    eps=1e-8
)
#to optimize the weights using epsilon value of 1e-8

In [35]:
epochs=10

scheduler=get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train)*epochs
)
#to modify the learning rate 
#num_training_steps will tell us how many time we want to update the learning rate

In [36]:
#definig the performance parameters
import numpy as np

In [37]:
from sklearn.metrics import f1_score

In [39]:
#pred=[0.9 0.05 0.05 0 0 0], this will be the ouput but we want
#preds=[1 0 0 0 0 0] output to be this

In [40]:
def f1_score_func(preds, labels):
    preds_flat=np.argmax(preds,axis=1).flatten()
    labels_flat=labels.flatten()
    return f1_score(labels_falt,preds_flat,average='weighted')
#we have flatten the predcitions to an arraya as well as the labels so that we never get a list of list
#then calculating the f1 score on flatten labels and predcitions
#using averages as weighted since we know that we have a imbalance in the classes

In [63]:
def accuracy_per_class(preds, labels):
    label_dict_inverse={v:k for k,v in label_dict.items()}
    preds_flat=np.argmax(preds,axis=1).flatten()
    labels_flat=labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds=preds_flat[labels_flat==label]
        y_true=labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')
#this will help to print the accuracy of per class with true label and predicted labels   

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [64]:
import random
#Creating our training loop
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [65]:
#to check if we are going to use gpu or cpu
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
#to shift the model to the device where we are using
print(device)

cpu


In [66]:
def evaluate(dataloader_val):

    model.eval()
    #don't do back propogation
    #it freezes all the weigths
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    #setting the model to training mode
    loss_train_total=0
    #initially setting loss to 0
    progress_bar=tqdm(dataloader_train,desc='Epoch{:1d}'.format(epoch),
                     leave=False,disable=False)
    #to show the training sequnece epochs
    for batch in progress_bar:
        
        model.zero_grad()
        #setting the initial gradients to zero
         
        batch=tuple(b.to(device) for b in batch)
        #to make sure that all the elements are in the same device
        
        inputs={
            'input_ids':batch[0],
            'attention_mask':batch[1],
            'labels':batch[2]
        }
        #the inputs that our training dataloader gave will be passed here
        
        output=model(**inputs)
        #inputing the whole data input list
        
        loss=output[0]
        loss_train_total+=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
        #clip our gradients to norm so that no problem of exploding and shrinking gradient will occur
        
        optimizer.step()
        scheduler.step()
        #stepping the optimzer and scheduler
        
        progress_bar.set_postfix('training_loss:{:.3f}'.format(loss.item()/len(batch)))
    
    #to save the model
    torch.save(model.state_dict(),f'Models/BERT_ft_epoch{epoch}.model')
    
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg=loss_train_totak/len(dataloader)
    
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss,predictions,true_vals=evaluate(dataloader_val)
    val_f1=f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted):{val_f1}')
                
               

In [52]:
#reloading the model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [53]:
model.to(device)
pass

In [56]:
model.load_state_dict(
    torch.load('Models/finetuned_bert_epoch_1_gpu_trained.model',
              map_location=torch.device('cpu')))
#loading the weigths back to cpu

<All keys matched successfully>

In [60]:
_,predictions,true_vals=evaluate(dataloader_val)
#we don't care of loss since we are not training

In [1]:
accuracy_per_class(predictions,true_vals)
#lets check the accuracy per class